In [1]:
import polars as pl

from usearch.index import Index, Matches
import numpy as np

In [2]:
q = pl.scan_csv(
    "./data/geonames/allCountries.txt",
    separator="\t",
    has_header=False,
    schema={
        "geonameid": pl.UInt32,
        "name": pl.Utf8,
        "asciiname": pl.Utf8,
        "alternatenames": pl.Utf8,
        "latitude": pl.Float32,
        "longitude": pl.Float32,
        "feature_class": pl.Utf8,
        "feature_code": pl.Utf8,
        "country_code": pl.Utf8,
        "cc2": pl.Utf8,
        "admin1_code": pl.Utf8,
        "admin2_code": pl.Utf8,
        "admin3_code": pl.Utf8,
        "admin4_code": pl.Utf8,
        "population": pl.Int64,
        "elevation": pl.Int32,
        "dem": pl.Int32,
        "timezone": pl.Utf8,
        "modification_date": pl.Datetime,
    },
)
df = q.collect()
df

geonameid,name,asciiname,alternatenames,latitude,longitude,feature_class,feature_code,country_code,cc2,admin1_code,admin2_code,admin3_code,admin4_code,population,elevation,dem,timezone,modification_date
u32,str,str,str,f32,f32,str,str,str,str,str,str,str,str,i64,i32,i32,str,datetime[μs]
2994701,"""Roc Meler""","""Roc Meler""","""Roc Mele,Roc Meler,Roc Mélé""",42.58765,1.7418,"""T""","""PK""","""AD""","""AD,FR""","""02""",null,null,null,0,2811,2348,"""Europe/Andorra""",2023-10-03 00:00:00
3017832,"""Pic de les Abelletes""","""Pic de les Abelletes""","""Pic de la Font-Negre,Pic de la…",42.525349,1.73343,"""T""","""PK""","""AD""","""FR""","""A9""","""66""","""663""","""66146""",0,null,2411,"""Europe/Andorra""",2014-11-05 00:00:00
3017833,"""Estany de les Abelletes""","""Estany de les Abelletes""","""Estany de les Abelletes,Etang …",42.529148,1.73362,"""H""","""LK""","""AD""","""FR""","""A9""",null,null,null,0,null,2260,"""Europe/Andorra""",2014-11-05 00:00:00
3023203,"""Port Vieux de la Coume d’Ose""","""Port Vieux de la Coume d'Ose""","""Port Vieux de Coume d'Ose,Port…",42.625679,1.61823,"""T""","""PASS""","""AD""",null,"""00""",null,null,null,0,null,2687,"""Europe/Andorra""",2014-11-05 00:00:00
3029315,"""Port de la Cabanette""","""Port de la Cabanette""","""Port de la Cabanette,Porteille…",42.599998,1.73333,"""T""","""PASS""","""AD""","""AD,FR""","""B3""","""09""","""091""","""09139""",0,null,2379,"""Europe/Andorra""",2014-11-05 00:00:00
…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…
12808977,"""Lilly Canyon""","""Lilly Canyon""",null,44.816669,-48.716671,"""U""","""CNYU""",null,null,"""00""",null,null,null,0,null,-9999,null,2024-05-15 00:00:00
12808978,"""Kettle Canyon""","""Kettle Canyon""",null,44.098888,-49.211681,"""U""","""CNYU""",null,null,"""00""",null,null,null,0,null,-9999,null,2024-05-15 00:00:00
12808979,"""Guy Canyon""","""Guy Canyon""",null,43.383331,-49.049999,"""U""","""CNYU""",null,null,"""00""",null,null,null,0,null,-9999,null,2024-05-15 00:00:00


In [3]:
my_coordinates1 = np.array([51.55, -0.1346], dtype=np.float32)
my_coordinates2 = np.array([37.77493, -122.41942], dtype=np.float32)

In [4]:
coordinates1 = df.select(["latitude", "longitude"]).to_numpy()
labels1 = df["geonameid"].to_numpy()

In [5]:
coordinates2 = np.zeros((df.shape[0], 2), dtype=np.float32)
coordinates2[:, 0] = df["latitude"].to_numpy()
coordinates2[:, 1] = df["longitude"].to_numpy()
labels2 = np.array(df["geonameid"], dtype=np.int32)

In [6]:
np.array_equal(coordinates1, coordinates2), np.array_equal(labels1, labels2)

(True, True)

In [7]:
index1: Index = Index(ndim=2, metric="haversine", dtype="f32")
index1.add(keys=labels1, vectors=coordinates1, log=True, copy=False)

Add: 100%|██████████| 12660854/12660854 [02:38<00:00, 79780.41vector/s]


array([ 2994701,  3017832,  3017833, ..., 12808979, 12808980, 12808981],
      dtype=uint64)

In [8]:
index2: Index = Index(ndim=2, metric="haversine", dtype="f32")
index2.add(keys=labels2, vectors=coordinates2, log=True, copy=False)

Add: 100%|██████████| 12660854/12660854 [02:03<00:00, 102679.19vector/s]


array([ 2994701,  3017832,  3017833, ..., 12808979, 12808980, 12808981],
      dtype=uint64)

In [9]:
output1: Matches = index1.search(
    vectors=my_coordinates1, count=10, exact=True, log=True
)
df.filter(pl.col("geonameid").is_in(output1.keys))

Search: 100%|██████████| 1/1 [00:01<00:00,  1.32s/vector]


geonameid,name,asciiname,alternatenames,latitude,longitude,feature_class,feature_code,country_code,cc2,admin1_code,admin2_code,admin3_code,admin4_code,population,elevation,dem,timezone,modification_date
u32,str,str,str,f32,f32,str,str,str,str,str,str,str,str,i64,i32,i32,str,datetime[μs]
12217233,"""Lazovský vrch""","""Lazovsky vrch""",null,49.26963,20.95969,"""T""","""MT""","""CF""",null,"""01""","""9143460""",null,null,0,859,853,"""Europe/Bratislava""",2021-02-26 00:00:00
9534996,"""Návary""","""Navary""",null,50.98085,15.20861,"""T""","""HLL""","""CG""",null,null,null,null,null,0,681,405,"""Africa/Brazzaville""",2015-05-05 00:00:00
12809069,"""Collège Albert-Camus""","""College Albert-Camus""",null,47.809559,3.57058,"""S""","""SCH""","""FR""",null,"""27""","""89""","""891""","""89024""",0,null,98,"""Europe/Paris""",2024-05-16 00:00:00
12624076,"""Fort Albert""","""Fort Albert""","""Fort Albert""",49.72633,-2.184,"""T""","""HLL""","""GG""",null,null,null,null,null,0,55,50,"""Europe/Guernsey""",2023-10-21 00:00:00
12749917,"""Istvánmező""","""Istvanmezo""",null,47.5047,19.095539,"""P""","""PPLX""","""HU""",null,"""05""",null,null,null,11794,null,103,"""Europe/Budapest""",2024-03-17 00:00:00
11154479,"""Erbach Schloss""","""Erbach Schloss""",null,49.657169,8.9925,"""S""","""CSTL""","""SO""",null,"""03""",null,null,null,0,null,216,"""Africa/Mogadishu""",2016-06-03 00:00:00
12809058,"""Lazarevka""","""Lazarevka""","""Lazarevka,Лазаревка""",50.342339,29.29698,"""P""","""PPLQ""","""UA""",null,"""27""","""1804""",null,null,0,null,170,"""Europe/Kyiv""",2024-05-16 00:00:00
7763448,"""Donnell Seamount""","""Donnell Seamount""",null,50.083328,-45.349998,"""U""","""SMU""",null,null,"""00""",null,null,null,0,null,-9999,null,2011-04-20 00:00:00
8617633,"""Coastal Waters Of Southeast Al…","""Coastal Waters Of Southeast Al…","""The Coastal Waters of Southeas…",52.829319,-130.913086,"""H""","""CHNM""",null,null,null,null,null,null,0,null,-9999,null,2013-10-01 00:00:00


In [10]:
output2: Matches = index2.search(
    vectors=my_coordinates1, count=10, exact=True, log=True
)
df.filter(pl.col("geonameid").is_in(output2.keys))

Search: 100%|██████████| 1/1 [00:01<00:00,  1.70s/vector]


geonameid,name,asciiname,alternatenames,latitude,longitude,feature_class,feature_code,country_code,cc2,admin1_code,admin2_code,admin3_code,admin4_code,population,elevation,dem,timezone,modification_date
u32,str,str,str,f32,f32,str,str,str,str,str,str,str,str,i64,i32,i32,str,datetime[μs]
6493096,"""Queens Hotel""","""Queens Hotel""",null,51.554901,-0.132,"""S""","""HTL""","""GB""",null,"""ENG""","""GLA""","""G3""",null,0,null,45,"""Europe/London""",2010-05-24 00:00:00
6690587,"""Kentish Town""","""Kentish Town""","""کینٹش ٹاؤن""",51.551682,-0.14125,"""P""","""PPL""","""GB""",null,"""ENG""","""GLA""","""C4""",null,0,null,40,"""Europe/London""",2011-03-03 00:00:00
6943739,"""The Corner House Hotel""","""The Corner House Hotel""",null,51.5485,-0.13006,"""S""","""HTL""","""GB""",null,"""ENG""","""GLA""","""C4""",null,0,null,51,"""Europe/London""",2010-05-25 00:00:00
6952900,"""Kentish Town Station""","""Kentish Town Station""","""Bahnhof Kentish Town,KTN,Kenti…",51.550831,-0.14083,"""S""","""RSTN""","""GB""",null,"""ENG""","""GLA""","""C4""",null,0,null,37,"""Europe/London""",2010-05-25 00:00:00
6954729,"""Kentish Town Underground Stati…","""Kentish Town Underground Stati…",null,51.55032,-0.13919,"""S""","""MTRO""","""GB""",null,"""ENG""","""GLA""","""C4""",null,0,null,38,"""Europe/London""",2010-05-25 00:00:00
7648321,"""Charlies Hotel""","""Charlies Hotel""",null,51.55526,-0.13072,"""S""","""HTL""","""GB""",null,"""ENG""","""GLA""","""G3""",null,0,null,44,"""Europe/London""",2011-01-31 00:00:00
9951999,"""Five Kings Hotel""","""Five Kings Hotel""",null,51.555279,-0.13074,"""S""","""HTL""","""GB""",null,"""ENG""","""GLA""","""G3""",null,0,null,44,"""Europe/London""",2015-01-10 00:00:00
9952020,"""Corner House Hotel Y Anexo""","""Corner House Hotel Y Anexo""",null,51.549252,-0.12864,"""S""","""HTL""","""GB""",null,"""ENG""","""GLA""","""G3""",null,0,null,51,"""Europe/London""",2015-01-10 00:00:00
10238344,"""Five Kings - Hsd""","""Five Kings - Hsd""",null,51.554981,-0.13261,"""S""","""HTL""","""GB""",null,"""ENG""","""GLA""","""G3""",null,0,null,44,"""Europe/London""",2015-04-28 00:00:00


In [11]:
print(coordinates1.flags)
print(coordinates2.flags)
print(labels1.flags)
print(labels2.flags)

  C_CONTIGUOUS : False
  F_CONTIGUOUS : True
  OWNDATA : False
  WRITEABLE : True
  ALIGNED : True
  WRITEBACKIFCOPY : False

  C_CONTIGUOUS : True
  F_CONTIGUOUS : False
  OWNDATA : True
  WRITEABLE : True
  ALIGNED : True
  WRITEBACKIFCOPY : False

  C_CONTIGUOUS : True
  F_CONTIGUOUS : True
  OWNDATA : False
  WRITEABLE : False
  ALIGNED : True
  WRITEBACKIFCOPY : False

  C_CONTIGUOUS : True
  F_CONTIGUOUS : True
  OWNDATA : True
  WRITEABLE : True
  ALIGNED : True
  WRITEBACKIFCOPY : False



In [13]:
coordinates1 = df.select(["latitude", "longitude"]).to_numpy(
    order="c"
)  # Change the array to be row-major
labels1 = df["geonameid"].to_numpy()

In [14]:
print(coordinates1.flags)
print(labels1.flags)

  C_CONTIGUOUS : True
  F_CONTIGUOUS : False
  OWNDATA : False
  WRITEABLE : True
  ALIGNED : True
  WRITEBACKIFCOPY : False

  C_CONTIGUOUS : True
  F_CONTIGUOUS : True
  OWNDATA : False
  WRITEABLE : False
  ALIGNED : True
  WRITEBACKIFCOPY : False



In [15]:
index1: Index = Index(ndim=2, metric="haversine", dtype="f32")
index1.add(keys=labels1, vectors=coordinates1, log=True, copy=False)

Add: 100%|██████████| 12660854/12660854 [01:54<00:00, 110576.58vector/s]


array([ 2994701,  3017832,  3017833, ..., 12808979, 12808980, 12808981],
      dtype=uint64)

In [16]:
output1: Matches = index1.search(
    vectors=my_coordinates1, count=10, exact=True, log=True
)
df.filter(pl.col("geonameid").is_in(output1.keys))

Search: 100%|██████████| 1/1 [00:00<00:00,  2.30vector/s]


geonameid,name,asciiname,alternatenames,latitude,longitude,feature_class,feature_code,country_code,cc2,admin1_code,admin2_code,admin3_code,admin4_code,population,elevation,dem,timezone,modification_date
u32,str,str,str,f32,f32,str,str,str,str,str,str,str,str,i64,i32,i32,str,datetime[μs]
6493096,"""Queens Hotel""","""Queens Hotel""",null,51.554901,-0.132,"""S""","""HTL""","""GB""",null,"""ENG""","""GLA""","""G3""",null,0,null,45,"""Europe/London""",2010-05-24 00:00:00
6690587,"""Kentish Town""","""Kentish Town""","""کینٹش ٹاؤن""",51.551682,-0.14125,"""P""","""PPL""","""GB""",null,"""ENG""","""GLA""","""C4""",null,0,null,40,"""Europe/London""",2011-03-03 00:00:00
6943739,"""The Corner House Hotel""","""The Corner House Hotel""",null,51.5485,-0.13006,"""S""","""HTL""","""GB""",null,"""ENG""","""GLA""","""C4""",null,0,null,51,"""Europe/London""",2010-05-25 00:00:00
6952900,"""Kentish Town Station""","""Kentish Town Station""","""Bahnhof Kentish Town,KTN,Kenti…",51.550831,-0.14083,"""S""","""RSTN""","""GB""",null,"""ENG""","""GLA""","""C4""",null,0,null,37,"""Europe/London""",2010-05-25 00:00:00
6954729,"""Kentish Town Underground Stati…","""Kentish Town Underground Stati…",null,51.55032,-0.13919,"""S""","""MTRO""","""GB""",null,"""ENG""","""GLA""","""C4""",null,0,null,38,"""Europe/London""",2010-05-25 00:00:00
7648321,"""Charlies Hotel""","""Charlies Hotel""",null,51.55526,-0.13072,"""S""","""HTL""","""GB""",null,"""ENG""","""GLA""","""G3""",null,0,null,44,"""Europe/London""",2011-01-31 00:00:00
9951999,"""Five Kings Hotel""","""Five Kings Hotel""",null,51.555279,-0.13074,"""S""","""HTL""","""GB""",null,"""ENG""","""GLA""","""G3""",null,0,null,44,"""Europe/London""",2015-01-10 00:00:00
9952020,"""Corner House Hotel Y Anexo""","""Corner House Hotel Y Anexo""",null,51.549252,-0.12864,"""S""","""HTL""","""GB""",null,"""ENG""","""GLA""","""G3""",null,0,null,51,"""Europe/London""",2015-01-10 00:00:00
10238344,"""Five Kings - Hsd""","""Five Kings - Hsd""",null,51.554981,-0.13261,"""S""","""HTL""","""GB""",null,"""ENG""","""GLA""","""G3""",null,0,null,44,"""Europe/London""",2015-04-28 00:00:00


In [17]:
output2: Matches = index2.search(
    vectors=my_coordinates1, count=10, exact=True, log=True
)
df.filter(pl.col("geonameid").is_in(output2.keys))

Search: 100%|██████████| 1/1 [00:01<00:00,  1.28s/vector]


geonameid,name,asciiname,alternatenames,latitude,longitude,feature_class,feature_code,country_code,cc2,admin1_code,admin2_code,admin3_code,admin4_code,population,elevation,dem,timezone,modification_date
u32,str,str,str,f32,f32,str,str,str,str,str,str,str,str,i64,i32,i32,str,datetime[μs]
6493096,"""Queens Hotel""","""Queens Hotel""",null,51.554901,-0.132,"""S""","""HTL""","""GB""",null,"""ENG""","""GLA""","""G3""",null,0,null,45,"""Europe/London""",2010-05-24 00:00:00
6690587,"""Kentish Town""","""Kentish Town""","""کینٹش ٹاؤن""",51.551682,-0.14125,"""P""","""PPL""","""GB""",null,"""ENG""","""GLA""","""C4""",null,0,null,40,"""Europe/London""",2011-03-03 00:00:00
6943739,"""The Corner House Hotel""","""The Corner House Hotel""",null,51.5485,-0.13006,"""S""","""HTL""","""GB""",null,"""ENG""","""GLA""","""C4""",null,0,null,51,"""Europe/London""",2010-05-25 00:00:00
6952900,"""Kentish Town Station""","""Kentish Town Station""","""Bahnhof Kentish Town,KTN,Kenti…",51.550831,-0.14083,"""S""","""RSTN""","""GB""",null,"""ENG""","""GLA""","""C4""",null,0,null,37,"""Europe/London""",2010-05-25 00:00:00
6954729,"""Kentish Town Underground Stati…","""Kentish Town Underground Stati…",null,51.55032,-0.13919,"""S""","""MTRO""","""GB""",null,"""ENG""","""GLA""","""C4""",null,0,null,38,"""Europe/London""",2010-05-25 00:00:00
7648321,"""Charlies Hotel""","""Charlies Hotel""",null,51.55526,-0.13072,"""S""","""HTL""","""GB""",null,"""ENG""","""GLA""","""G3""",null,0,null,44,"""Europe/London""",2011-01-31 00:00:00
9951999,"""Five Kings Hotel""","""Five Kings Hotel""",null,51.555279,-0.13074,"""S""","""HTL""","""GB""",null,"""ENG""","""GLA""","""G3""",null,0,null,44,"""Europe/London""",2015-01-10 00:00:00
9952020,"""Corner House Hotel Y Anexo""","""Corner House Hotel Y Anexo""",null,51.549252,-0.12864,"""S""","""HTL""","""GB""",null,"""ENG""","""GLA""","""G3""",null,0,null,51,"""Europe/London""",2015-01-10 00:00:00
10238344,"""Five Kings - Hsd""","""Five Kings - Hsd""",null,51.554981,-0.13261,"""S""","""HTL""","""GB""",null,"""ENG""","""GLA""","""G3""",null,0,null,44,"""Europe/London""",2015-04-28 00:00:00
